<a href="https://colab.research.google.com/github/IvanSerem/ds2-proyecto-agricola/blob/main/DataScienceII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analizar eventos climáticos relevantes para el rendimiento agrícola en Argentina y su impacto potencial en los cultivos.

Hoy aplicaremos un flujo de trabajo completo que simula un proyecto real de ciencia de datos y los preparará para su Tercera Entrega. Nuestra misión es transformar un gran volumen de datos crudos en una historia clara y con insights valiosos.

El plan de hoy es el siguiente:

Adquisición de Datos: Partiremos de los datos masivos que obtendremos de la API del USGS, que nos entregó un DataFrame crudo con más de 8,000 terremotos y 30 columnas.

Data Wrangling (Limpieza y Enriquecimiento): Veremos que los datos crudos no son prácticos para el análisis. Realizaremos un proceso de limpieza para seleccionar, renombrar y transformar las columnas más importantes, creando un dataset final, limpio y potente.

EDA y Storytelling (Análisis y Visualización): Con nuestro dataset ya preparado, formularemos preguntas de interés y las responderemos creando visualizaciones efectivas para contar una historia.

¿Por Qué No Usamos Todas las Columnas? La Necesidad de Seleccionar
Al ejecutar .info() sobre nuestro df_terremotos_crudo, vemos una lista de 30 columnas como esta:

1.1. Importación de Librerías y Definición de Parámetros

In [45]:
# Importamos las librerías esenciales
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

# Configuramos un estilo visual para todos nuestros gráficos
sns.set_theme(style="whitegrid")

# Definimos los parámetros de la petición
url_base = "https://power.larc.nasa.gov/api/temporal/daily/point"
años_a_consultar = list(range(2014, 2024))


1.2. Extracción de Datos en Lotes

In [48]:
import pandas as pd
import requests
import time

# =========================
# Configuración base NASA POWER
# =========================
url_base = "https://power.larc.nasa.gov/api/temporal/daily/point"
años_a_consultar = list(range(2014, 2024))

# Provincias argentinas (capital provincial como punto representativo)
provincias = {
    "Buenos Aires": (-34.61, -58.38),
    "Catamarca": (-28.47, -65.78),
    "Chaco": (-27.45, -58.99),
    "Chubut": (-43.30, -65.10),
    "Córdoba": (-31.42, -64.18),
    "Corrientes": (-27.47, -58.83),
    "Entre Ríos": (-31.73, -60.53),
    "Formosa": (-26.18, -58.17),
    "Jujuy": (-24.18, -65.33),
    "La Pampa": (-36.62, -64.29),
    "La Rioja": (-29.41, -66.86),
    "Mendoza": (-32.89, -68.83),
    "Misiones": (-27.37, -55.90),
    "Neuquén": (-38.95, -68.06),
    "Río Negro": (-40.81, -63.00),
    "Salta": (-24.78, -65.41),
    "San Juan": (-31.53, -68.52),
    "San Luis": (-33.30, -66.34),
    "Santa Cruz": (-51.62, -69.22),
    "Santa Fe": (-31.63, -60.70),
    "Santiago del Estero": (-27.78, -64.27),
    "Tierra del Fuego": (-54.80, -68.30),
    "Tucumán": (-26.82, -65.22)
}

# Acá guardamos TODO (todas las provincias, todos los años)
lista_dataframes = []

print("--- Realizando la petición en lotes (por provincia y por año) con NASA POWER ---")

for provincia, (lat, lon) in provincias.items():
    print(f"\n📍 Provincia: {provincia} | lat={lat}, lon={lon}")

    for anio in años_a_consultar:
        print(f"  Obteniendo datos para el año {anio}...")

        params = {
            "latitude": lat,
            "longitude": lon,
            "start": f"{anio}0101",
            "end": f"{anio}1231",
            "parameters": "T2M,T2M_MAX,T2M_MIN,PRECTOTCORR,WS2M,RH2M",
            "community": "AG",
            "format": "JSON"
        }

        try:
            response = requests.get(url_base, params=params, timeout=60)
            response.raise_for_status()

            payload = response.json()

            # NASA POWER no trae "data": trae "properties/parameter"
            data_params = payload["properties"]["parameter"]

            # data_params: {variable: {YYYYMMDD: valor}}
            df_anual = pd.DataFrame(data_params)

            # convertir índice YYYYMMDD a fecha
            df_anual["date"] = pd.to_datetime(df_anual.index, format="%Y%m%d")
            df_anual["anio"] = anio
            df_anual["provincia"] = provincia
            df_anual["lat"] = lat
            df_anual["lon"] = lon

            lista_dataframes.append(df_anual.reset_index(drop=True))
            print(f"    ✅ Datos {provincia} {anio}: {len(df_anual)} registros.")

            time.sleep(0.2)

        except requests.exceptions.RequestException as e:
            print(f"    ❌ Error en {provincia} {anio}: {e}")


--- Realizando la petición en lotes (por provincia y por año) con NASA POWER ---

📍 Provincia: Buenos Aires | lat=-34.61, lon=-58.38
  Obteniendo datos para el año 2014...
    ✅ Datos Buenos Aires 2014: 365 registros.
  Obteniendo datos para el año 2015...
    ✅ Datos Buenos Aires 2015: 365 registros.
  Obteniendo datos para el año 2016...
    ✅ Datos Buenos Aires 2016: 366 registros.
  Obteniendo datos para el año 2017...
    ✅ Datos Buenos Aires 2017: 365 registros.
  Obteniendo datos para el año 2018...
    ✅ Datos Buenos Aires 2018: 365 registros.
  Obteniendo datos para el año 2019...
    ✅ Datos Buenos Aires 2019: 365 registros.
  Obteniendo datos para el año 2020...
    ✅ Datos Buenos Aires 2020: 366 registros.
  Obteniendo datos para el año 2021...
    ✅ Datos Buenos Aires 2021: 365 registros.
  Obteniendo datos para el año 2022...
    ✅ Datos Buenos Aires 2022: 365 registros.
  Obteniendo datos para el año 2023...
    ✅ Datos Buenos Aires 2023: 365 registros.

📍 Provincia: Cat

1.3. Inspeccionando los Resultados del Bucle

In [49]:
# Verificamos cuántos DataFrames hemos recolectado
# Debería ser uno por cada año que no dio error.
print(f"Se han recolectado {len(lista_dataframes_anuales)} DataFrames en total.")

# Inspeccionemos el primer DataFrame de la lista para ver su estructura y contenido
# Este corresponde a los datos del primer año que consultamos (2020)
if lista_dataframes_anuales:
    print("\n--- Analizando el PRIMER DataFrame de la lista (datos del año 2020) ---")

    # Seleccionamos el primer elemento de la lista
    primer_df = lista_dataframes_anuales[0]

    # Mostramos su forma y sus primeras filas
    print(f"Forma del primer DataFrame: {primer_df.shape}")
    print("Primeras 3 filas:")
    display(primer_df.head(3))

Se han recolectado 10 DataFrames en total.

--- Analizando el PRIMER DataFrame de la lista (datos del año 2020) ---
Forma del primer DataFrame: (365, 8)
Primeras 3 filas:


,T2M,T2M_MAX,T2M_MIN,PRECTOTCORR,WS2M,RH2M,date,anio
0,27.94,30.52,25.09,23.96,2.83,65.29,2014-01-01,2014
1,24.30,25.51,22.09,14.94,6.22,71.09,2014-01-02,2014
2,21.41,23.97,18.59,0.00,5.18,54.43,2014-01-03,2014


1.4. Consolidación y Verificación Finalmente, unimos todos los DataFrames de nuestra lista en una única tabla maestra con pd.concat y realizamos una primera verificación.

In [50]:
# 1.4. Consolidación y Verificación

# Verificamos que la lista no esté vacía antes de continuar
if lista_dataframes:
    # Unimos todos los DataFrames (todas las provincias y años)
    df_clima_crudo = pd.concat(lista_dataframes, ignore_index=True)

    print("\n--- ¡DataFrame Consolidado Creado Exitosamente! ---")
    print(f"Forma del DataFrame final: {df_clima_crudo.shape}")

    # Información general para un primer diagnóstico
    print("\n--- Información General ---")
    df_clima_crudo.info()

    # Verificación rápida de cobertura
    print("\n--- Verificaciones básicas ---")
    print("Provincias incluidas:", df_clima_crudo["provincia"].nunique())
    print("Años incluidos:", df_clima_crudo["anio"].unique())
    print("Rango de fechas:", df_clima_crudo["date"].min(), "→", df_clima_crudo["date"].max())
else:
    print("No se pudieron cargar los datos.")



--- ¡DataFrame Consolidado Creado Exitosamente! ---
Forma del DataFrame final: (83996, 11)

--- Información General ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83996 entries, 0 to 83995
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   T2M          83996 non-null  float64       
 1   T2M_MAX      83996 non-null  float64       
 2   T2M_MIN      83996 non-null  float64       
 3   PRECTOTCORR  83996 non-null  float64       
 4   WS2M         83996 non-null  float64       
 5   RH2M         83996 non-null  float64       
 6   date         83996 non-null  datetime64[ns]
 7   anio         83996 non-null  int64         
 8   provincia    83996 non-null  object        
 9   lat          83996 non-null  float64       
 10  lon          83996 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(1), object(1)
memory usage: 7.0+ MB

--- Verificaciones básicas ---
Provincias incluidas

“A diferencia del dataset de terremotos, el dataset climático proveniente de NASA POWER no presenta estructuras anidadas ni metadata técnica redundante. Por lo tanto, el proceso de data wrangling se enfocó principalmente en la selección de variables climáticas relevantes, la estandarización de nombres y la creación de nuevas variables temporales y derivadas.”

In [61]:
# ============================================
# MÓDULO 2 – DATA WRANGLING
# Dataset Climático – NASA POWER
# ============================================

import pandas as pd

# ------------------------------------------------
# 1. PARTIMOS DEL DATAFRAME CRUDO
# ------------------------------------------------
# Se asume que df_clima_crudo ya fue creado en el Módulo 1
# (adquisición y consolidación de datos)

print("Forma del DataFrame crudo:")
print(df_clima_crudo.shape)

# ------------------------------------------------
# 2. SELECCIÓN DE COLUMNAS RELEVANTES
# ------------------------------------------------
columnas_relevantes = [
    "T2M",
    "T2M_MAX",
    "T2M_MIN",
    "PRECTOTCORR",
    "WS2M",
    "RH2M",
    "date",
    "anio",
    "provincia",
    "lat",
    "lon"
]

df_clima_limpio = df_clima_crudo[columnas_relevantes].copy()

print("\nColumnas seleccionadas:")
print(df_clima_limpio.columns)

# ------------------------------------------------
# 3. RENOMBRADO DE COLUMNAS (CLARIDAD SEMÁNTICA)
# ------------------------------------------------
df_clima_limpio = df_clima_limpio.rename(columns={
    "T2M": "temp_media",
    "T2M_MAX": "temp_max",
    "T2M_MIN": "temp_min",
    "PRECTOTCORR": "precipitacion",
    "WS2M": "vel_viento",
    "RH2M": "humedad",
    "date": "fecha",
    "anio": "anio",
    "provincia": "provincia",
    "lat": "latitud",
    "lon": "longitud"
})

# ------------------------------------------------
# 4. VERIFICACIÓN GENERAL DEL DATASET
# ------------------------------------------------
print("\n--- Información del DataFrame Limpio ---")
df_clima_limpio.info()

print("\n--- Primeras filas del DataFrame Limpio ---")
df_clima_limpio.head()


Forma del DataFrame crudo:
(83996, 11)

Columnas seleccionadas:
Index(['T2M', 'T2M_MAX', 'T2M_MIN', 'PRECTOTCORR', 'WS2M', 'RH2M', 'date',
       'anio', 'provincia', 'lat', 'lon'],
      dtype='object')

--- Información del DataFrame Limpio ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83996 entries, 0 to 83995
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   temp_media     83996 non-null  float64       
 1   temp_max       83996 non-null  float64       
 2   temp_min       83996 non-null  float64       
 3   precipitacion  83996 non-null  float64       
 4   vel_viento     83996 non-null  float64       
 5   humedad        83996 non-null  float64       
 6   fecha          83996 non-null  datetime64[ns]
 7   anio           83996 non-null  int64         
 8   provincia      83996 non-null  object        
 9   latitud        83996 non-null  float64       
 10  longitud       83996 non-nul

,temp_media,temp_max,temp_min,precipitacion,vel_viento,humedad,fecha,anio,provincia,latitud,longitud
0,27.94,30.52,25.09,23.96,2.83,65.29,2014-01-01,2014,Buenos Aires,-34.61,-58.38
1,24.30,25.51,22.09,14.94,6.22,71.09,2014-01-02,2014,Buenos Aires,-34.61,-58.38
2,21.41,23.97,18.59,0.00,5.18,54.43,2014-01-03,2014,Buenos Aires,-34.61,-58.38
3,24.35,28.11,20.04,0.00,5.16,47.29,2014-01-04,2014,Buenos Aires,-34.61,-58.38
4,26.02,30.74,20.78,0.00,3.16,61.97,2014-01-05,2014,Buenos Aires,-34.61,-58.38


In [73]:
df_clima_limpio["vel_viento_kmh"] = (
    df_clima_limpio["vel_viento"] * 3.6
).round(1)


In [72]:
df_clima_limpio.head()


,temp_media,temp_max,temp_min,precipitacion,vel_viento,humedad,fecha,anio,provincia,latitud,longitud,vel_viento_kmh
0,27.94,30.52,25.09,23.96,2.83,65.29,2014-01-01,2014,Buenos Aires,-34.61,-58.38,10.2
1,24.30,25.51,22.09,14.94,6.22,71.09,2014-01-02,2014,Buenos Aires,-34.61,-58.38,22.4
2,21.41,23.97,18.59,0.00,5.18,54.43,2014-01-03,2014,Buenos Aires,-34.61,-58.38,18.6
3,24.35,28.11,20.04,0.00,5.16,47.29,2014-01-04,2014,Buenos Aires,-34.61,-58.38,18.6
4,26.02,30.74,20.78,0.00,3.16,61.97,2014-01-05,2014,Buenos Aires,-34.61,-58.38,11.4
